In [60]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import itertools
import time
import random
from sklearn.preprocessing import StandardScaler

import os
import sys
sys.path.append('../src')

from math_utils import *
from model import *
from utils import *

In [61]:
DATA_PATH = '../Concrete_Data.xls' # link dataset: https://archive.ics.uci.edu/dataset/165/concrete+compressive+strength
SEED = 42
df_data = pd.read_excel(DATA_PATH).sample(frac=1, random_state=SEED)

In [62]:
df_data

,Cement (component 1)(kg in a m^3 mixture),Blast Furnace Slag (component 2)(kg in a m^3 mixture),Fly Ash (component 3)(kg in a m^3 mixture),Water (component 4)(kg in a m^3 mixture),Superplasticizer (component 5)(kg in a m^3 mixture),Coarse Aggregate (component 6)(kg in a m^3 mixture),Fine Aggregate (component 7)(kg in a m^3 mixture),Age (day),"Concrete compressive strength(MPa, megapascals)"
31,266.00,114.0,0.00,228.00,0.00,932.0,670.00,365,52.908320
109,362.60,189.0,0.00,164.90,11.60,944.7,755.80,7,55.895819
136,389.90,189.0,0.00,145.90,22.00,944.7,755.80,28,74.497882
88,362.60,189.0,0.00,164.90,11.60,944.7,755.80,3,35.301171
918,145.00,0.0,179.00,202.00,8.00,824.0,869.00,28,10.535193
...,...,...,...,...,...,...,...,...,...
87,286.30,200.9,0.00,144.70,11.20,1004.6,803.70,3,24.400556
330,246.83,0.0,125.08,143.30,11.99,1086.8,800.89,14,42.216615
466,190.34,0.0,125.18,166.61,9.88,1079.0,798.90,100,33.563692
121,475.00,118.8,0.00,181.10,8.90,852.1,781.50,28,68.299493


In [63]:
data = df_data.to_numpy() #converting from excel to numpy array(meanwhile shuffling)

dev_len, test_len = 0.8*(len(data)), 0.2*(len(data)) # 80% for dev and 20% for test
dev_set, test_set = data[:int(dev_len)], data[int(dev_len):]

train_len, val_len = 0.8*(len(dev_set)), 0.2*(len(dev_set)) # 80% for train and 20% for validation
train_set, val_set = dev_set[:int(train_len)], dev_set[int(train_len):]

In [64]:
scaler = StandardScaler()
# Standardizing the features
train_set[:, :-1] = scaler.fit_transform(train_set[:, :-1])
val_set[:, :-1] = scaler.transform(val_set[:, :-1])
test_set[:, :-1] = scaler.transform(test_set[:, :-1])
dev_set[:, :-1] = scaler.transform(dev_set[:, :-1])

In [65]:
len(train_set), len(val_set), len(test_set) # 80% for train, 10% for validation and 10% for test

(659, 165, 206)

## Model Selection

In [66]:
def grid_search(config_dict:dict, train_set:np.ndarray, val_set:np.ndarray, config_trials:int=5, metric:callable=mse) -> tuple:
    """
    Perform grid search over the given configuration dictionary.

    Parameters
    ----------
    config_dict : dict
        Dictionary containing the configurations to be tested.
    config_trials : int
        Number of trials for each configuration.

    Returns
    -------
    config_dict : dataframe
        all configurations and their mean and variance of MSE
    """

    metric_name = str(metric.__name__) if hasattr(metric, '__name__') else str(metric)

    df_dict = {'hidden_size': [], 'activation': [], 'init_method': [], 'init_params': [], 'tikhonov': [],
               f'train_{metric_name}': [], f'train_{metric_name}_var': [], f'val_{metric_name}': [], f'val_{metric_name}_var': []}
    # create all possible combination of the configurations
    keys, values = zip(*config_dict.items())
    configs = [dict(zip(keys, v)) for v in itertools.product(*values)]
    for config in configs:
        df_dict['hidden_size'].append(config['hidden_size'])
        df_dict['activation'].append(str(config['activation'].__name__))
        df_dict['init_method'].append(config['init_method'])
        df_dict['init_params'].append(config['init_params'])
        df_dict['tikhonov'].append(config['tikhonov'])

        val_values = []
        train_values = []
        for _ in range(config_trials):
            model = ELM(input_size=config['input_size'], hidden_size=config['hidden_size'], hidden_activation=config['activation'], 
                        init_method=config['init_method'], init_params=config['init_params'])
            model.fit(train_set[:, :-1], train_set[:, -1].reshape(-1, 1), config['tikhonov'])

            pred = model.predict(train_set[:, :-1])
            train_values.append(metric(train_set[:, -1], pred))

            pred = model.predict(val_set[:, :-1])
            val_values.append(metric(val_set[:, -1], pred))

        df_dict[f'train_{metric_name}'].append(np.mean(train_values))
        df_dict[f'train_{metric_name}_var'].append(np.var(train_values))
        df_dict[f'val_{metric_name}'].append(np.mean(val_values))
        df_dict[f'val_{metric_name}_var'].append(np.var(val_values))

    
    config_df = pd.DataFrame(df_dict)
    return config_df


In [76]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

config_dict = {
    'input_size': [8],
    'hidden_size': [2048],
    'activation': [np.tanh],
    'init_method': ['uniform'],
    'init_params': [(-1, 1)],
    'tikhonov': [1e-6]
}

results = grid_search(config_dict, train_set, val_set, config_trials=1, metric=mae)

In [78]:
results = results.sort_values(by=f'val_{str(mae.__name__)}', ascending=True)
results

,hidden_size,activation,init_method,init_params,tikhonov,train_mae,train_mae_var,val_mae,val_mae_var
0,2048,tanh,uniform,"(-1, 1)",0.000001,2.582122,0.0,4.408818,0.0


In [52]:
config_dict = {
    'input_size': [8],
    'hidden_size': [100, 200, 300, 400, 500],
    'activation': [np.tanh, sigmoid],
    'init_method': ['normal'],
    'init_params': [(0, 0.1), (0, 0.5), (0, 1)]
}

results = grid_search(config_dict, train_set, val_set, config_trials=5, metric=mae)

In [18]:
results = results.sort_values(by=f'val_{mae.__name__}', ascending=True)
results

,hidden_size,activation,init_method,init_params,tikhonov,train_mae,train_mae_var,val_mae,val_mae_var
1,300,<ufunc 'tanh'>,uniform,"(-0.4, 0.4)",0.000001,4.106910,0.010138,4.630509,0.000180
0,300,<ufunc 'tanh'>,uniform,"(-0.5, 0.5)",0.000001,4.155889,0.021352,4.762776,0.010143
2,300,<function sigmoid at 0x7f60c4131a20>,uniform,"(-0.5, 0.5)",0.000001,4.627652,0.000850,4.862812,0.000104
3,300,<function sigmoid at 0x7f60c4131a20>,uniform,"(-0.4, 0.4)",0.000001,4.936368,0.004631,4.939689,0.000021


## Test Assessment

In [59]:
model = ELM(input_size=8, hidden_size=100, hidden_activation=np.tanh, init_method='normal', init_params=(0, 0.1))
model.fit(dev_set[:, :-1], dev_set[:, -1].reshape(-1, 1))
pred = model.predict(test_set[:, :-1])
print(f'Test MAE: {mae(test_set[:, -1], pred)}')

Test MAE: 21.771596006473366
